In [ ]:
!pip install transformers torch scikit-learn pandas

In [ ]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report

In [ ]:
data=pd.read_excel('/content/Emotions_DS.xlsx')

In [ ]:
# Extract columns
# Assuming columns are named as follows: 'Utterance', 'Dialogue_Act', 'Emotion', 'Type'
utterances = data['Utterance']
dialogue_acts = data['Dialogue_Act']
emotions = data['Emotion']
types = data['Type']

In [ ]:
# Tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# One-hot encode Dialogue Acts (59 unique dialogue acts)
#Removed sparse argument. It is no longer supported
dialogue_act_encoder = OneHotEncoder(handle_unknown='ignore')
dialogue_act_encoded = dialogue_act_encoder.fit_transform(data[['Dialogue_Act']])

In [ ]:
# Label encode Emotion (7 unique emotions)
emotion_encoder = LabelEncoder()
emotion_labels = emotion_encoder.fit_transform(data['Emotion'])

In [ ]:
# Optionally, One-hot encode the 'Type' if needed
#Removed sparse argument. It is no longer supported and will cause an error
type_encoder = OneHotEncoder(handle_unknown='ignore')
type_encoded = type_encoder.fit_transform(data[['Type']])

In [ ]:
# Tokenize utterances
def tokenize_utterance(utterance):
    if not isinstance(utterance, str):
        # Handle the case where utterance is not a string
        # For instance, you could try converting it to a string or skip it
        if isinstance(utterance, list):
            utterance = ' '.join(utterance) # Join list elements if utterance is a of list type
        else:
            return None # or handle it in a way that makes sense for your data
    return tokenizer(utterance, padding='max_length', truncation=True, return_tensors="pt", max_length=128)

tokenizer: This is a pre-defined function or object (not shown in this snippet) that performs the actual tokenization. It converts the text into tokens that can be used by models.

padding='max_length': This ensures that the tokenized output is padded to a fixed length (128 in this case) regardless of the length of the input. Padding ensures consistency in token sequence length.


truncation=True: If the input text is longer than the max_length, it will be truncated to fit the specified length (128 tokens in this case).


return_tensors="pt": This specifies that the output should be returned as PyTorch tensors, suitable for use with models based on PyTorch.


max_length=128: This defines the maximum length of the tokenized output. Any text longer than 128 tokens will be truncated.

In [ ]:
data['tokenized'] = data['Utterance'].apply(tokenize_utterance)

In [ ]:
# Example: show tokenized data
print(data['tokenized'].head())

0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
3    [input_ids, token_type_ids, attention_mask]
4    [input_ids, token_type_ids, attention_mask]
Name: tokenized, dtype: object


In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, dataframe, dialogue_act_encoded, emotion_labels, type_encoded):
        self.data = dataframe
        self.dialogue_act_encoded = dialogue_act_encoded
        self.emotion_labels = emotion_labels
        self.type_encoded = type_encoded

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        if row['tokenized'] is not None:
            input_ids = row['tokenized']['input_ids'].squeeze(0)
            attention_mask = row['tokenized']['attention_mask'].squeeze(0)
        else:
            input_ids = torch.zeros(128, dtype=torch.long)  # or handle it in a way that makes sense for your data
            attention_mask = torch.zeros(128, dtype=torch.long)


        dialogue_act = torch.tensor(self.dialogue_act_encoded[idx].toarray(), dtype=torch.float)
        emotion = torch.tensor(self.emotion_labels[idx], dtype=torch.long)
        type_feat = torch.tensor(self.type_encoded[idx].toarray(), dtype=torch.float)  # Convert to dense array before creating tensor

        return input_ids, attention_mask, dialogue_act, emotion, type_feat

In [ ]:
class EmotionDetectionModel(nn.Module):
    def __init__(self, num_dialogue_act_features, num_type_features):
        super(EmotionDetectionModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc1 = nn.Linear(768 + num_dialogue_act_features + num_type_features, 256)  # Include both dialogue act and type features
        self.fc2 = nn.Linear(256, len(emotion_encoder.classes_))  # Predict based on 7 unique emotions
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask, dialogue_act, type_feat):
        # Get BERT embeddings
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = bert_outputs.pooler_output  # [CLS] token representation

        # Combine BERT output with dialogue act and type features
        combined_input = torch.cat((cls_output, dialogue_act.squeeze(1), type_feat.squeeze(1)), dim=1)

        # Pass through fully connected layers
        x = self.fc1(combined_input)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x  # Output emotion class probabilities

In [ ]:
# Hyperparameters
EPOCHS = 4
BATCH_SIZE = 32
LEARNING_RATE = 1e-5

In [ ]:
# Prepare DataLoader
dataset = EmotionDataset(data, dialogue_act_encoded, emotion_labels, type_encoded)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Model, Loss, Optimizer
model = EmotionDetectionModel(num_dialogue_act_features=dialogue_act_encoded.shape[1],num_type_features=type_encoded.shape[1])
criterion = nn.CrossEntropyLoss()  # Since we are predicting emotion classes
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

EmotionDetectionModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids, attention_mask, dialogue_act, emotion, type_feat = [x.to(device) for x in batch]

        # Ensure type_feat has only two dimensions
        type_feat = torch.squeeze(type_feat, 1) # remove dimension at position 1 if it is of size 1

        # Ensure dialogue_act has the correct number of dimensions
        if dialogue_act.dim() == 3:
            dialogue_act = dialogue_act.squeeze(1)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, dialogue_act, type_feat)

        # Adjust the outputs to be 2D if necessary
        if outputs.dim() == 3:
            outputs = outputs[:, -1, :]  # Get the last time step output, if applicable

        loss = criterion(outputs, emotion)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(dataloader)}')

Epoch 1/4, Loss: nan
Epoch 2/4, Loss: nan
Epoch 3/4, Loss: nan
Epoch 4/4, Loss: nan


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, dialogue_act, emotion, type_feat = [x.to(device) for x in batch]
            outputs = model(input_ids, attention_mask, dialogue_act, type_feat)
            _, preds = torch.max(outputs, dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(emotion.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=[str(i) for i in emotion_encoder.classes_]))

In [ ]:
evaluate(model, dataloader)

Classification Report:
              precision    recall  f1-score   support

          -3       0.03      1.00      0.06       166
          -2       0.00      0.00      0.00       177
          -1       0.00      0.00      0.00       727
           0       0.00      0.00      0.00      4098
           1       0.00      0.00      0.00        64
           2       0.00      0.00      0.00        13
           9       0.00      0.00      0.00         2

    accuracy                           0.03      5247
   macro avg       0.00      0.14      0.01      5247
weighted avg       0.00      0.03      0.00      5247



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
